*Luis Antonio Ortega Andrés    
Antonio Coín Castro*

# Aprendizaje por Refuerzo

Segunda parte de la práctica     
16 Mayo 2021

## Enunciado
El siguiente notebook implementa un entorno con un brazo robótico de 2 motores. Las acciones posibles son 3*3 (número de acciones en motor1 * número de acciones en motor 2).

Se deberá cambiar y extender el código para implementar un sistema de control óptimo donde el extremo del brazo llegue lo más rápidamente al target (cuadrado azul). El cuadrado rojo puede estar en cualquier posición de la rejilla.

Se implementará un sistema basado en Q-learning o SARSA donde **la tabla es sustituida por una red neuronal**.

## Cuestiones

### Pregunta 1.
*¿Qué has necesitado cambiar en la definición del entorno?*

Se han hecho una serie de modificaciones sobre el código inicial, que comentamos a continuación.

**Sobre el objetivo del juego**

Se ha modificado la meta: para considerar que hemos ganado en un episodio, el brazo debe permanecer un número de pasos seguidos dentro del objetivo, no solo tocarlo.

**Sobre las variables de estado**

- Hemos fijado la variable `state_dim` a 7, que es el número de variables de estado que tiene nuestro agente (ver pregunta 2).
- Hemos añadido una variable `steps_in_target` que lleva la cuenta del número de pasos seguidos que lleva el brazo dentro del objetivo. También hemos añadido una variable `threshold` que controla el número mínimo de pasos que debe permanecer el brazo dentro del objetivo para considerar que se ha cumplido la tarea (ver preguntas 2 y 3).
- Hemos añadido una variable `norm_dist` que fijamos a la mitad de la longitud del escenario (es decir, a 200). Esto nos servirá para normalizar las distancias y hacer que estén en la misma escala, cosa que es importante de cara a utilizarlas como entrada en la red neuronal.

**Sobre el método `reset`**

Hemos implementado distintos modos de juego, dependiendo de cómo se reinicia la posición del brazo y del objetivo.

- `supereasy`: no se resetea nada. La idea era que el brazo aprendiera a quedarse dentro del objetivo sin salirse.
- `easy`: se resetea la posición del brazo a su posición inicial. Pretendíamos que el brazo aprendiese a ir a un sitio concreto lo más rápido posible.
- `medium`: se resetea la posición del brazo a una posición aleatoria. La idea es similar al modo anterior, pero intentando que aprenda a llegar a un objetivo partiendo de cualquier sitio.
- `hard`: se resetea la posición del *target* a una posición aleatoria (pero alcanzable por el brazo). Este es el modo por defecto y el que consideramos como objetivo de la práctica.
- `follow`: se desplaza ligeramente la posición del *target*. La idea aquí era simular el comportamiento de un humano desplazando el objetivo con el ratón, de forma que el brazo aprendiera a seguirlo suavemente.

**Sobre el método `render`**

Hemos modificado este método y el correspondiente de la clase `Viewer`, para que este último reciba las coordenadas del objetivo y pueda volver
a pintarlo tras cada reinicio aleatorio.

### Pregunta 2.
*¿Cómo has formalizado el estado S?*

Hemos considerado necesarias 7 variables de estado:

- El primer grupo de variables comprende **la posición del objetivo**, representada por la distancia vectorial del mismo al centro del tablero.
- El segundo grupo de variables hace referencia a **la posición del antebrazo con respecto al objetivo**. Esto lo medimos como la distancia vectorial del extremo del antebrazo al objetivo.
- El tercer grupo de variables hace referencia a **la posición del codo con respecto al objetivo**, medida de nuevo con la distancia vectorial. Estas dos variables no son estrictamente necesarias, pero hemos observado que mejora la estabilidad del entrenamiento (ver pregunta 5).
- Finalmente, consideramos una variable binaria (0-1) que simbolice **si el brazo está dentro del objetivo o no** en el paso actual.

Notamos que, como comentábamos antes, todas las distancias se encuentran normalizadas y en la misma escala.

### Pregunta 3.
*¿Cómo has diseñado la recompensa R?*

La idea principal de la recompensa es considerar la distancia euclídea entre el objetivo y el extremo más lejano del brazo. En primer lugar, la normalizamos con la misma constante que las variables de estado, y después le cambiamos el signo, de forma que a menor distancia, mayor sea la recompensa.

Además de usar la distancia como tal, aumentamos significativamente la recompensa cuando el brazo toque el objetivo para incentivar que lo haga.
Se puede comprobar que la recompensa hasta ahora se encuentra en $[-\sqrt{2}, 0]$, por lo que decidimos sumar $\sqrt{2}$ a la recompensa si se toca el objetivo. De esta forma, una posición perfecta (el brazo en el centro del objetivo) obtendría una recompensa de $\sqrt{2}$, que tiene la misma magnitud que la peor recompensa posible ($-\sqrt{2}$). Además, así nos aseguramos que la recompensa es positiva si y solo si se toca el objetivo.

Finalmente, consideramos premiar en gran medida si el brazo consigue mantenerse durante un número de pasos seguidos dentro del objetivo. Concretamente, hemos considerado que se acaba el episodio si se mantiene durante 10 pasos seguidos, y en ese caso sumamos 10 a la recompensa.

*Nota:* la manera que tenemos realmente de medir si el brazo está dentro del objetivo es comprobar si se encuentra en el círculo inscrito al cuadrado azul, lo que nos parece una aproximación suficientemente buena y que simplifica los cálculos.

### Pregunta 4.
*¿Qué parámetros te han dado mejores resultados?*

La combinación de parámetros que ha proporcionado los mejores resultados es la siguiente:

- Entrenar el modelo durante 200 episodios$^{(\ast)}$, limitando el número de pasos en cada episodio a 400. De esta forma no tarda demasiado en los episodios iniciales.
- Utilizar como red neuronal una red con una capa oculta totalmente conectada de 200 neuronas y activación ReLU, seguida de otra capa completamente conectada con 9 neuronas de salida (tantas como acciones distintas) y activación lineal.
- El valor de $\gamma=0.9$ que hemos usado en muchos ejemplos de clase.
- El optimizador Adam con *learning rate* de $0.001$ y pérdida *'mse'*. También hemos probado el SGD con un *learning rate* mayor, pero los resultados no eran mejores.
- Utilizar un valor inicial de $\varepsilon=0.7$, e irlo disminuyendo con un *decay* exponencial multiplicando por $0.99$ tras cada episodio, de forma que al final tenía un valor en torno a $0.1$.
- En cuanto a la memoria, la hemos construido con tamaño máximo de 5000, entrenando con un tamaño de *batch* de 100 muestras aleatorias cada 200 pasos (es decir, dos veces por episodio).

$(\ast)$ A la hora de ejecutar el código hemos hecho primero 100 episodios y después otros 100, para comprobar cómo iba el entrenamiento. El tiempo total consumido para los 200 episodios ha sido de aproximadamente 1 hora, utilizando la GPU de Google Colab.

Durante el entrenamiento no siempre se alcanza el objetivo en cada episodio, ya que la exploración en la elección de acciones está activa (aunque en los últimos episodios ya se observa que la mayoría de veces lo consigue). Tras el entrenamiento, hemos probado el modelo desactivando completamente la exploración, y ha alcanzado el objetivo en 36 pasos, consiguiendo una recompensa total de 6.52. Si lo ejecutamos en local con el entorno de visualización activado, podemos ver que el brazo ha aprendido a seguir al objetivo de manera aceptable.

### Pregunta 5.
*¿Qué podrías hacer para minimizar las "vibraciones" y/o maximizar la "suavidad" en el control del brazo robótico?*

Por un lado, hemos observado que añadiendo al estado información sobre la articulación del codo se reducen un poco las vibraciones. Como decíamos, esta información no es estrictamente necesaria para que la red aprenda (al principio estábamos probando sin ellas y funcionaba medianamente bien), pero contribuye a suavizar el movimiento. Probablemente podríamos reducir aún más las vibraciones teniendo en cuenta también los ángulos de ambas articulaciones. 

Por otro lado, en las primeras pruebas vimos que aunque el brazo conseguía tocar el objetivo, no aprendía a quedarse dentro y oscilaba alrededor. Esto lo arreglamos en gran medida modificando la tarea y haciendo que tuviera que estar una serie de pasos seguidos dentro para ganar, adaptando también las variables de estado y la recompensa. Así el movimiento es mucho más suave y se reducen las vibraciones notablemente.

## Viewer

In [2]:
import pyglet
import numpy as np


class Viewer(pyglet.window.Window):
    color = {"background": [1] * 3 + [1]}
    fps_display = pyglet.clock.Clock()
    bar_thc = 5

    def __init__(
        self,
        width,
        height,
        arm1_coords,
        arm2_coords,
        arm1_ang,
        arm2_ang,
        target_coords,
        target_width,
        mouse_in,
    ):
        super(Viewer, self).__init__(
            width, height, resizable=False, caption="Arm", vsync=False
        )  # vsync=False to not use the monitor FPS
        self.set_location(x=80, y=10)
        pyglet.gl.glClearColor(*self.color["background"])

        self.arm1_coords = arm1_coords
        self.arm2_coords = arm2_coords
        self.arm1_ang = arm1_ang
        self.arm2_ang = arm2_ang

        self.target_coords = target_coords
        self.mouse_in = mouse_in
        self.target_width = target_width

        self.center_coords = np.array((min(width, height) / 2,) * 2)
        self.batch = pyglet.graphics.Batch()

        arm1_box, arm2_box, target_box = [0] * 8, [0] * 8, [0] * 8
        c1, c2, c3 = (249, 86, 86) * 4, (86, 109, 249) * 4, (249, 39, 65) * 4
        self.target = self.batch.add(
            4, pyglet.gl.GL_QUADS, None, ("v2f", target_box), ("c3B", c2)
        )
        self.arm1 = self.batch.add(
            4, pyglet.gl.GL_QUADS, None, ("v2f", arm1_box), ("c3B", c1)
        )
        self.arm2 = self.batch.add(
            4, pyglet.gl.GL_QUADS, None, ("v2f", arm2_box), ("c3B", c1)
        )

    def render(self, arm1_coords, arm2_coords, arm1_ang, arm2_ang, target_coords):
        self.arm1_coords = arm1_coords
        self.arm2_coords = arm2_coords
        self.arm1_ang = arm1_ang
        self.arm2_ang = arm2_ang
        self.target_coords = target_coords

        pyglet.clock.tick()
        self._update_arm()
        self.switch_to()
        self.dispatch_events()
        self.dispatch_event("on_draw")
        self.flip()

    def on_draw(self):
        self.clear()
        self.batch.draw()
        # self.fps_display.draw()

    def _update_arm(self):
        target_width = self.target_width
        target_coords = self.target_coords
        target_box = (
            target_coords[0] - target_width,
            target_coords[1] - target_width,
            target_coords[0] + target_width,
            target_coords[1] - target_width,
            target_coords[0] + target_width,
            target_coords[1] + target_width,
            target_coords[0] - target_width,
            target_coords[1] + target_width,
        )
        self.target.vertices = target_box

        # (x0, y0, x1, y1)
        arm1_aux = np.hstack((self.center_coords, self.arm1_coords))
        # (x1, y1, x2, y2)
        arm2_aux = np.hstack((self.arm1_coords, self.arm2_coords))

        arm1_thick_rad = np.pi / 2 - self.arm1_ang
        x01 = arm1_aux[0] - np.cos(arm1_thick_rad) * self.bar_thc
        y01 = arm1_aux[1] + np.sin(arm1_thick_rad) * self.bar_thc

        x02 = arm1_aux[0] + np.cos(arm1_thick_rad) * self.bar_thc
        y02 = arm1_aux[1] - np.sin(arm1_thick_rad) * self.bar_thc

        x11 = arm1_aux[2] + np.cos(arm1_thick_rad) * self.bar_thc
        y11 = arm1_aux[3] - np.sin(arm1_thick_rad) * self.bar_thc

        x12 = arm1_aux[2] - np.cos(arm1_thick_rad) * self.bar_thc
        y12 = arm1_aux[3] + np.sin(arm1_thick_rad) * self.bar_thc
        arm1_box = (x01, y01, x02, y02, x11, y11, x12, y12)

        arm2_thick_rad = np.pi / 2 - (self.arm1_ang + self.arm2_ang)
        x11_ = arm2_aux[0] + np.cos(arm2_thick_rad) * self.bar_thc
        y11_ = arm2_aux[1] - np.sin(arm2_thick_rad) * self.bar_thc

        x12_ = arm2_aux[0] - np.cos(arm2_thick_rad) * self.bar_thc
        y12_ = arm2_aux[1] + np.sin(arm2_thick_rad) * self.bar_thc

        x21 = arm2_aux[2] - np.cos(arm2_thick_rad) * self.bar_thc
        y21 = arm2_aux[3] + np.sin(arm2_thick_rad) * self.bar_thc

        x22 = arm2_aux[2] + np.cos(arm2_thick_rad) * self.bar_thc
        y22 = arm2_aux[3] - np.sin(arm2_thick_rad) * self.bar_thc
        arm2_box = (x11_, y11_, x12_, y12_, x21, y21, x22, y22)

        self.arm1.vertices = arm1_box
        self.arm2.vertices = arm2_box

    def on_key_press(self, symbol, modifiers):
        if symbol == pyglet.window.key.UP:
            self.arm1_ang += 0.1 * 10
            print(
                self.arm1_coords - self.target_coords,
                self.arm1_ang,
                self.arm2_coords - self.target_coords,
                self.arm2_ang,
            )
        elif symbol == pyglet.window.key.DOWN:
            self.arm1_ang -= 0.1 * 10
            print(
                self.arm1_coords - self.target_coords,
                self.arm1_ang,
                self.arm2_coords - self.target_coords,
                self.arm2_ang,
            )
        elif symbol == pyglet.window.key.LEFT:
            self.arm2_ang += 0.1
            print(
                self.arm1_coords - self.target_coords,
                self.arm1_ang,
                self.arm2_coords - self.target_coords,
                self.arm2_ang,
            )
        elif symbol == pyglet.window.key.RIGHT:
            self.arm2_ang -= 0.1
            print(
                self.arm1_coords - self.target_coords,
                self.arm1_ang,
                self.arm2_coords - self.target_coords,
                self.arm2_ang,
            )
        elif symbol == pyglet.window.key.Q:
            pyglet.clock.set_fps_limit(1000)
        elif symbol == pyglet.window.key.A:
            pyglet.clock.set_fps_limit(30)

    def on_mouse_motion(self, x, y, dx, dy):
        self.target_coords[:] = [x, y]

    def on_mouse_enter(self, x, y):
        self.mouse_in[0] = True

    def on_mouse_leave(self, x, y):
        self.mouse_in[0] = False

## Agent

In [3]:
import gym
import numpy as np
from gym.spaces import Discrete, Box


class myEnv(gym.Env):
    metadata = {"render.modes": ["human", "rgb_array"],
                "video.frames_per_second": 50}

    def __init__(self, threshold=10, target_coords=[250, 300], mode="hard"):
        self.state_dim = 7
        self.dt = 0.1  # refresh rate

        self.arm1_long = 100
        self.arm2_long = 100
        self.arm1_ang = 0
        self.arm2_ang = 0
        self.arm1_coords = np.array([0, 0])
        self.arm2_coords = np.array([0, 0])

        self.viewer = None
        self.viewer_xy = (400, 400)
        self.norm_dist = self.viewer_xy[0]/2.
        self.got_target = False
        self.steps_in_target = 0
        self.threshold = threshold
        self.mouse_in = np.array([False])
        self.target_width = 15

        self.action_space = Discrete(3 * 3)
        self.observation_space = Box(
            low=-1e10, high=1e10, shape=(self.state_dim,), dtype=np.float32
        )
        self.observation_space.n = self.state_dim
        self.mode = mode

        self.target_coords = np.array(target_coords)
        self.target_coords_init = self.target_coords.copy()
        self.center_coords = np.array(self.viewer_xy) / 2

    def render(self, mode="human"):
        if self.viewer is None:
            self.viewer = Viewer(
                *self.viewer_xy,
                self.arm1_coords,
                self.arm2_coords,
                self.arm1_ang,
                self.arm2_ang,
                self.target_coords,
                self.target_width,
                self.mouse_in
            )
        self.viewer.render(
            self.arm1_coords, self.arm2_coords, self.arm1_ang,
            self.arm2_ang, self.target_coords
        )

    def reset(self):
        if self.mode == "supereasy":  # Do not reset anything
            pass
        elif self.mode == "easy":  # Reset arm to (0, 0)
            self.arm1_ang = 0
            self.arm2_ang = 0
            self.arm1_coords = np.array([0, 0])
            self.arm2_coords = np.array([0, 0])
        elif self.mode == 'medium':  # Reset arm to random position
            self.arm1_ang = np.random.uniform(0, 2*np.pi)
            self.arm2_ang = np.random.uniform(0, 2*np.pi)
            self.arm1_coords = np.random.randint(low=100, high=300, size=2)
            self.arm2_coords = np.random.randint(low=100, high=300, size=2)
        elif self.mode == "hard":  # Move target to random position
            pxy = np.random.randint(low=100, high=300, size=2)
            self.target_coords[:] = pxy
        elif self.mode == "follow":  # Move target to random nearby position
            self.target_coords += np.random.randint(low=-20, high=20, size=2)
            self.target_coords = np.clip(self.target_coords, 100, 300)

        s = self._get_state()
        self.steps_in_target = 0
        self.got_target = False

        return s

    def _get_state(self):
        """Returns internal state."""
        target_pos = (self.target_coords - self.center_coords)/self.norm_dist
        arm1_dist = (self.target_coords - self.arm1_coords)/self.norm_dist
        arm2_dist = (self.target_coords - self.arm2_coords)/self.norm_dist
        has_target = 1.0 if self.steps_in_target > 0 else 0.0
        return np.array([*target_pos, *arm1_dist, *arm2_dist, has_target])

    def step(self, act):
        action1 = act // 3 - 1
        action2 = act % 3 - 1

        self.arm1_ang += action1 * self.dt
        self.arm1_ang %= np.pi * 2
        self.arm2_ang += action2 * self.dt
        self.arm2_ang %= np.pi * 2

        self.arm1_coords = self.center_coords.copy()
        self.arm1_coords[0] += self.arm1_long * np.cos(self.arm1_ang)
        self.arm1_coords[1] += self.arm1_long * np.sin(self.arm1_ang)

        self.arm2_coords = self.arm1_coords.copy()
        self.arm2_coords[0] += self.arm2_long * \
            np.cos(self.arm1_ang + self.arm2_ang)
        self.arm2_coords[1] += self.arm2_long * \
            np.sin(self.arm1_ang + self.arm2_ang)

        s = self._get_state()

        # Euclidean distance between target and arm2
        dist = np.linalg.norm(self.arm2_coords - self.target_coords)
        r = -dist/self.norm_dist

        # Modify reward
        if dist <= self.target_width:  # if touching the target
            r += np.sqrt(2)
            self.steps_in_target += 1
            if self.steps_in_target > self.threshold:
                r += 10
                self.got_target = True
        else:
            self.steps_in_target = 0
            self.got_target = False

        return s, r, self.got_target, {}

    def close(self):
        if self.viewer:
            self.viewer.close()

## Deep Q-Learning

In [4]:
import numpy as np
import time
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from collections import deque
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


class Memory():
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = []

    def append(self, item):
        self.memory.append(item)
        if self.len() > self.max_memory:
            del self.memory[0]

    def len(self):
        return len(self.memory)


def build_q_nnet(n_state, n_actions, alpha, lr_decay):
    model = Sequential()
    model.add(Dense(200, input_shape=(n_state,),
                    bias_initializer="RandomNormal",
                    activation="relu"))
    model.add(Dense(n_actions))

    if lr_decay:
        lr = ExponentialDecay(
            alpha,
            decay_steps=1000,
            decay_rate=0.9,
            staircase=True)
    else:
        lr = alpha

    model.compile(loss='mse', optimizer=Adam(lr=lr))

    return model


def replay(q_nnet, gamma, mem, n_actions, batch_size=100):
    if batch_size <= 0:
        return q_nnet

    n_samples = min(batch_size, mem.len())
    minibatch = random.sample(mem.memory, n_samples)
    X_train = np.zeros((n_samples, minibatch[0][0].shape[0]))
    y_train = np.zeros((n_samples, n_actions))

    for i, (S, A, R, S_, done) in enumerate(minibatch):
        if not done:
            # next state is not terminal:
            q_target = R + gamma*q_nnet.predict(S_.reshape(1, -1)).max()
        else:
            q_target = R  # next state is terminal

        target_vector = q_nnet.predict(np.array(S.reshape(1, -1)))
        target_vector[0][A] = q_target
        q_nnet.fit(np.array(S.reshape(1, -1)),
                   target_vector, verbose=0)  # update

    return q_nnet


def choose_action(state, nnet, epsilon):
    state_actions = nnet.predict(state.reshape(1, -1))[0]
    # act non-greedy or state-action have no value
    if np.random.uniform() < epsilon or (state_actions == 0).all():
        act = np.random.randint(len(state_actions))
    else:   # act greedy
        act = np.argmax(state_actions)
    return act


def rl(env, epsilon, alpha, gamma, n_episodes,
       sleep_time=0.001, vis=True, max_steps=1000,
       decay=0.99, min_epsilon=0.01, max_memory=5000, n_memory=500,
       batch_size=100, q_nnet=None, lr_decay=False):
    """Perform reinforcement learning."""
    n_actions = env.action_space.n
    n_states = env.state_dim

    if q_nnet is None:
        q_nnet = build_q_nnet(n_states, n_actions, alpha, lr_decay)

    mem = Memory(max_memory)
    steps_hist = []
    rewards_hist = []

    for episode in range(n_episodes):
        S = env.reset()
        done = False
        total_reward = 0

        for step in range(max_steps):
            act = choose_action(S, q_nnet, epsilon)
            # take action & get next state and reward
            S_, R, done, _ = env.step(act)
            total_reward += R
            mem.append([S, act, R, S_, done])
            S = S_  # move to next state

            if (step + 1) % n_memory == 0:
                q_nnet = replay(q_nnet, gamma, mem, n_actions, batch_size)

            if vis and not done:
                if (step + 1) % 100 == 0:
                    print(f"[Episodio {episode + 1}] Step {step + 1}")
                env.render()
                time.sleep(sleep_time)

            if done:
                steps_hist.append(step + 1)
                rewards_hist.append(total_reward)
                print('Episodio %s: total steps = %s, total reward = %.2f' % (
                    episode + 1, step + 1, total_reward))
                break

            elif step + 1 == max_steps:
                print(
                    f"Episodio {episode + 1}: no alcanza tras {step + 1} pasos. Total reward = {total_reward:.2f}")
                
        if epsilon > min_epsilon:
            epsilon *= decay

    return q_nnet, episode + 1, steps_hist, rewards_hist

## Entrenamiento

In [21]:
env = myEnv(mode='hard')

# Load pre-trained weights, or set to 'None' for building a new model
model = load_model('q_nnet-200.h5')

q_nnet, episodes, steps, rewards = rl(
    env,
    epsilon=0.7,
    alpha=0.001,
    lr_decay=False,
    gamma=0.9,
    n_episodes=100,
    min_epsilon=0.1,
    vis=False,
    decay=0.99,
    max_steps=400,
    max_memory=5000,
    n_memory=200,
    batch_size=100,
    q_nnet=model,
)

Episodio 1: no alcanza tras 400 pasos. Total reward = -358.75
Episodio 2: no alcanza tras 400 pasos. Total reward = -147.48
Episodio 3: no alcanza tras 400 pasos. Total reward = -93.25
Episodio 4: no alcanza tras 400 pasos. Total reward = -95.16
Episodio 5: no alcanza tras 400 pasos. Total reward = -42.86
Episodio 6: no alcanza tras 400 pasos. Total reward = -160.89
Episodio 7: no alcanza tras 400 pasos. Total reward = -136.29
Episodio 8: no alcanza tras 400 pasos. Total reward = -225.05
Episodio 9: no alcanza tras 400 pasos. Total reward = -339.29
Episodio 10: no alcanza tras 400 pasos. Total reward = -46.14
Episodio 11: total steps = 278, total reward = 57.36
Episodio 12: no alcanza tras 400 pasos. Total reward = -166.13
Episodio 13: no alcanza tras 400 pasos. Total reward = -190.78
Episodio 14: no alcanza tras 400 pasos. Total reward = -80.13
Episodio 15: total steps = 137, total reward = 45.41
Episodio 16: no alcanza tras 400 pasos. Total reward = 38.05
Episodio 17: no alcanza tras

In [22]:
# Save weights
q_nnet.save('q_nnet-200.h5')

## Validación

In [ ]:
def test_policy(env, q_nnet, vis=True,
                sleep_time=0.001,
                max_steps=1000, verbose=True):
    S = env.reset()
    total_reward = 0

    for step in range(max_steps):
        act = np.argmax(q_nnet.predict(S.reshape(1, -1))[0])
        S_, R, done, _ = env.step(act)
        total_reward += R
        S = S_  # move to next state

        if vis and not done:
            if step + 1 % 100 == 0 and verbose:
                print(f"[test_policy] Step {step + 1}")
            env.render()
            time.sleep(sleep_time)

        if done:
            if verbose:
                print(f"[test_policy] Alcanza objetivo tras {step + 1} pasos")
            break
        elif step + 1 == max_steps and verbose:
            print(f"[test_policy] No alcanza objetivo tras {step + 1} pasos")

    return total_reward, done

In [25]:
q_nnet = load_model('q_nnet-200.h5')
env2 = myEnv(mode='hard')
total_R, _ = test_policy(env2, q_nnet, vis=False)
print(f"Refuerzo total política: {total_R:.2f}")

[test_policy] Alcanza objetivo tras 36 pasos
Refuerzo total política: 6.52
